#### Reference:
- https://learn.deeplearning.ai/courses/building-coding-agents-with-tool-execution/

## Overview: 
- This notebook creates a Data Analyst Agent 🤓

First start by filtering warnings and loading environment variables.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from helper import load_env
load_env()

Now, let's create the data analyst agent. You will create a sandbox, then open the `pokemon.csv` dataset and write it to the sandbox as `data.csv`.

In [3]:
from lib.utils import create_sandbox

sbx = create_sandbox()

with open("pokemon.csv", "r") as f:
    content = f.read()

sbx.files.write("data.csv", content)

INFO     [sandbox] 🚀 Creating new Sandbox.create(id=ilg2a1wiyvz6p665507k4)

WriteInfo(name='data.csv', type='file', path='/home/user/data.csv')

Now you can import your `coding_agent` and modify the system prompt to let it know that the user has uploaded a dataset called `data.csv` and it should create interesting plots.

In [4]:
from lib.coding_agent import coding_agent, log
from lib.tools_schemas import execute_code_schema
from lib.tools import execute_code
from openai import OpenAI

client = OpenAI()

system = """You are a senior python programmer. 
You must run the code using the `execute_code` tool.
The user has uploaded a data.csv.
You help the user understanding the data 
by creating interesting plots.
"""

tools = { "execute_code" : execute_code }

Now you can query the data as normal and the sandbox will execute code on the data to answer the queries.

In [5]:
messages = []

query = "What is the data about?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

╭───────────────────────────────────────────────── 🤖 Tool Call ──────────────────────────────────────────────────╮
│ execute_code                                                                                                    │
│ Arguments: {"code":"import pandas as pd\ndf = pd.read_csv('data.csv')\ndf.head()"}                              │
│ Result: {"results": [{"text": "                     abilities  against_bug  against_dark  against_dragon  \\\n0 │
│ ['Overgrow', 'Chlorophyll']          1.0           1.0             1.0   \n1  ['Overgrow', 'Chl...              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO     ✨: The data appears to be about Pokémon. It contains various attributes for each Pokémon, including:     
                                                                                                                   
         - Abilities                                                                                               
         - Effectiveness against different types (e.g., bug, dark, dragon)                                         
         - Base stats such as attack, defense, hp (hit points), special attack, special defense, and speed         
         - Classification and type information                                                                     
         - Physical attributes like height and weight                                                              
         - Other attributes like the generation they belong to, whether they are legendary, and their Pokédex      
         number                                                                                                    
                                                                                                                   
         Would you like me to create some interesting plots to explore this data further?

INFO     [agent] 🔢 tokens: 4013 total

In [6]:
query = "Can you aggregate the pokemons by type?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

╭───────────────────────────────────────────────── 🤖 Tool Call ──────────────────────────────────────────────────╮
│ execute_code                                                                                                    │
│ Arguments: {"code":"# Group by 'type1' and aggregate with mean for numerical columns\n# Choosing type1 as the   │
│ main type for aggregation\nagg_by_type = df.groupby('type1').mean(numeric_only=True)\n\n# Show the fi...        │
│ Result: {"results": [{"text": "          against_bug  against_dark  against_dragon  against_electric  \\\ntype1 │
│ \nbug          0.826389      0...                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO     ✨: I have aggregated the Pokémon data by their primary type (type1). The aggregation computes the mean of
         various numerical attributes for each type. This includes stats like attack, defense, speed, weight,      
         height, and also effectiveness against different types, among many others.                                
                                                                                                                   
         Would you like to see any specific plots or analyses based on these aggregated values? For example, we can
         compare average stats by type or effectiveness against certain types.

INFO     [agent] 🔢 tokens: 8812 total

### Gradio UI

You can use this provided Gradio UI to have a nicer user experience when chatting with your code generation agent. It will also give interesting information about the context stack, including the number of tokens used in the result, by the assistant, the user, and tools.

In [7]:
from lib.ui import ui

ui(coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

* Running on local URL:  https://0.0.0.0:7861
* Running on public URL: https://15e4cc71b4a0ea71a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Your Turn

Now it's your turn to explore this unknown dataset. Ask questions and learn more about the information inside.

In [8]:
sbx = create_sandbox()
with open("unknown.csv", "rb") as f:
    content = f.read()
sbx.files.write("data.csv", content)

messages = []

ui(
    coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

INFO     [sandbox] 🔌 Reconnecting to Sandbox.create(id=ilg2a1wiyvz6p665507k4)

* Running on local URL:  https://0.0.0.0:7862
* Running on public URL: https://d26a31b2323b871fbb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
